In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/opt/anaconda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from .models import AttentionResNetCifar10

In [3]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_datagen.fit(x_train)
val_datagen.fit(x_train)

In [5]:
# build a model
model = AttentionResNetCifar10(n_classes=10)

In [ ]:
# plot model graph
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [6]:
# prepare usefull callbacks
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=7, min_lr=10e-7, epsilon=0.01, verbose=1)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=1)
callbacks= [lr_reducer, early_stopper]

In [7]:
# define loss, metrics, optimizer
model.compile(keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fits the model on batches with real-time data augmentation
batch_size = 32

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)//batch_size, epochs=200,
                    validation_data=val_datagen.flow(x_test, y_test, batch_size=batch_size), 
                    validation_steps=len(x_test)//batch_size,
                    callbacks=callbacks, initial_epoch=0)

Epoch 1/200
1562/1562 [==============================] - 304s 195ms/step - loss: 1.9663 - acc: 0.3429 - val_loss: 1.5818 - val_acc: 0.4564
Epoch 2/200
1562/1562 [==============================] - 458s 293ms/step - loss: 1.5777 - acc: 0.4458 - val_loss: 1.5661 - val_acc: 0.4970
Epoch 3/200
1562/1562 [==============================] - 428s 274ms/step - loss: 1.4027 - acc: 0.5012 - val_loss: 1.4493 - val_acc: 0.5338
Epoch 4/200
1562/1562 [==============================] - 429s 275ms/step - loss: 1.2705 - acc: 0.5515 - val_loss: 1.3063 - val_acc: 0.5899
Epoch 5/200
1562/1562 [==============================] - 438s 280ms/step - loss: 1.1694 - acc: 0.5890 - val_loss: 1.0526 - val_acc: 0.6367
Epoch 6/200
1562/1562 [==============================] - 460s 295ms/step - loss: 1.0992 - acc: 0.6160 - val_loss: 0.9479 - val_acc: 0.6646
Epoch 7/200
1562/1562 [==============================] - 428s 274ms/step - loss: 1.0190 - acc: 0.6436 - val_loss: 1.1057 - val_acc: 0.6377
Epoch 8/200
1562/1562 [====

In [13]:
model.evaluate_generator(val_datagen.flow(x_test, y_test), steps=len(x_test)/32, use_multiprocessing=True)

[0.3466754733324051, 0.8917]